In [64]:
import numpy as np
import pandas as pd

import lazypredict
from lazypredict.Supervised import LazyClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [41]:
df = pd.read_csv("Dataset/Tree_Data.csv")
df.head()

,No,Plot,Subplot,Species,Light_ISF,Light_Cat,Core,Soil,Adult,Sterile,...,AMF,EMF,Phenolics,Lignin,NSC,Census,Time,Event,Harvest,Alive
0,126,1,C,Acer saccharum,0.11,Med,2017,Prunus serotina,I,Non-Sterile,...,22.00,NaN,-0.56,13.86,12.15,4,14.00,1.00,NaN,NaN
1,11,1,C,Quercus alba,0.11,Med,2017,Quercus rubra,970,Non-Sterile,...,15.82,31.07,5.19,20.52,19.29,33,115.50,0.00,NaN,X
2,12,1,C,Quercus rubra,0.11,Med,2017,Prunus serotina,J,Non-Sterile,...,24.45,28.19,3.36,24.74,15.01,18,63.00,1.00,NaN,NaN
3,2823,7,D,Acer saccharum,0.08,Med,2016,Prunus serotina,J,Non-Sterile,...,22.23,NaN,-0.71,14.29,12.36,4,14.00,1.00,NaN,NaN
4,5679,14,A,Acer saccharum,0.06,Low,2017,Prunus serotina,689,Non-Sterile,...,21.15,NaN,-0.58,10.85,11.20,4,14.00,1.00,NaN,NaN


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2783 entries, 0 to 2782
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   No           2783 non-null   int64  
 1   Plot         2783 non-null   int64  
 2   Subplot      2783 non-null   object 
 3   Species      2783 non-null   object 
 4   Light_ISF    2783 non-null   float64
 5   Light_Cat    2783 non-null   object 
 6   Core         2783 non-null   int64  
 7   Soil         2783 non-null   object 
 8   Adult        2783 non-null   object 
 9   Sterile      2783 non-null   object 
 10  Conspecific  2783 non-null   object 
 11  Myco         2783 non-null   object 
 12  SoilMyco     2783 non-null   object 
 13  PlantDate    2783 non-null   object 
 14  AMF          2783 non-null   float64
 15  EMF          1283 non-null   float64
 16  Phenolics    2783 non-null   float64
 17  Lignin       2783 non-null   float64
 18  NSC          2783 non-null   float64
 19  Census

In [43]:
df.nunique()

No             2783
Plot             18
Subplot           5
Species           4
Light_ISF        53
Light_Cat         3
Core              2
Soil              7
Adult            36
Sterile           2
Conspecific       3
Myco              2
SoilMyco          3
PlantDate        19
AMF             924
EMF             682
Phenolics       494
Lignin         1095
NSC             998
Census           22
Time             22
Event             2
Harvest           1
Alive             1
dtype: int64

In [44]:
df.isnull().sum()

No                0
Plot              0
Subplot           0
Species           0
Light_ISF         0
Light_Cat         0
Core              0
Soil              0
Adult             0
Sterile           0
Conspecific       0
Myco              0
SoilMyco          0
PlantDate         0
AMF               0
EMF            1500
Phenolics         0
Lignin            0
NSC               0
Census            0
Time              0
Event             1
Harvest        2079
Alive          2292
dtype: int64

In [45]:
len(df)

2783

In [46]:
df = df.drop(columns=["No", "EMF", "Harvest", "Alive", "PlantDate"])

In [47]:
df["Event"].isna()

0       False
1       False
2       False
3       False
4       False
        ...  
2778    False
2779    False
2780    False
2781     True
2782    False
Name: Event, Length: 2783, dtype: bool

In [48]:
df = df.dropna(subset=["Event"])

In [49]:
#df = df.fillna(0)
#df.head()

In [50]:
df.corr()

,No,Plot,Light_ISF,Core,AMF,Phenolics,Lignin,NSC,Census,Time,Event
No,1.00,1.00,0.25,0.04,0.08,0.04,-0.01,0.03,-0.06,-0.06,-0.07
Plot,1.00,1.00,0.25,0.04,0.08,0.04,-0.01,0.03,-0.06,-0.06,-0.07
Light_ISF,0.25,0.25,1.00,0.05,0.10,0.10,0.08,0.23,0.02,0.02,-0.05
Core,0.04,0.04,0.05,1.00,-0.06,0.05,0.09,0.05,0.05,0.05,-0.05
AMF,0.08,0.08,0.10,-0.06,1.00,-0.12,-0.30,-0.18,-0.09,-0.09,0.13
Phenolics,0.04,0.04,0.10,0.05,-0.12,1.00,0.77,0.79,0.38,0.38,-0.58
Lignin,-0.01,-0.01,0.08,0.09,-0.30,0.77,1.00,0.55,0.29,0.29,-0.52
NSC,0.03,0.03,0.23,0.05,-0.18,0.79,0.55,1.00,0.37,0.37,-0.41
Census,-0.06,-0.06,0.02,0.05,-0.09,0.38,0.29,0.37,1.00,1.00,-0.23
Time,-0.06,-0.06,0.02,0.05,-0.09,0.38,0.29,0.37,1.00,1.00,-0.23


In [51]:
y = df["Event"]
X = df.drop(columns=["Event"])

In [59]:
#X.iloc[:, 0]

0        126
1         11
2         12
3       2823
4       5679
        ... 
2777    7086
2778    7165
2779    7217
2780    7306
2782    7401
Name: No, Length: 2782, dtype: int64

In [62]:
#for i in range(X.shape[1]):
    #print(df.columns[i])
    #np.corrcoef(X.iloc[:, i], y)

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.head()

,No,Plot,Subplot,Species,Light_ISF,Light_Cat,Core,Soil,Adult,Sterile,Conspecific,Myco,SoilMyco,PlantDate,AMF,Phenolics,Lignin,NSC,Census,Time
361,2279,6,C,Prunus serotina,0.09,Med,2017,Sterile,H,Sterile,Sterilized,AMF,Sterile,6/5/18,8.73,0.06,7.69,11.93,7,24.50
1845,5173,12,B,Prunus serotina,0.06,Low,2016,Quercus rubra,1688,Non-Sterile,Heterospecific,AMF,EMF,6/7/18,21.36,0.68,6.47,10.46,12,42.00
1337,7348,18,A,Acer saccharum,0.14,High,2016,Quercus alba,1478,Non-Sterile,Heterospecific,AMF,EMF,6/4/18,16.14,0.31,14.16,13.01,7,24.50
1601,4575,11,B,Acer saccharum,0.06,Low,2017,Quercus rubra,970,Non-Sterile,Heterospecific,AMF,EMF,6/11/18,13.31,0.01,10.11,10.79,11,38.50
2600,6673,16,C,Quercus alba,0.13,High,2016,Populus grandidentata,285,Non-Sterile,Heterospecific,EMF,EMF,5/29/18,30.28,5.26,18.88,23.64,15,52.50


In [70]:
#scaler = StandardScaler().fit(X_train)
#X_train_scaled = scaler.transform(X_train)
#X_test_scaled = scaler.transform(X_test)

In [68]:
clf_laz = LazyClassifier(verbose=0, ignore_warnings=True)
lazy_model, lazy_pred_str = clf_laz.fit(X_train, X_test, y_train, y_test)
lazy_model

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:02<00:00, 10.22it/s]

[LightGBM] [Info] Number of positive: 1185, number of negative: 901
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000149 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1504
[LightGBM] [Info] Number of data points in the train set: 2086, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.568073 -> initscore=0.273993
[LightGBM] [Info] Start training from score 0.273993
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.97,0.97,0.97,0.97,0.17
RandomForestClassifier,0.97,0.97,0.97,0.97,0.24
BaggingClassifier,0.97,0.97,0.97,0.97,0.07
XGBClassifier,0.97,0.97,0.97,0.97,0.11
AdaBoostClassifier,0.96,0.96,0.96,0.96,0.16
ExtraTreesClassifier,0.96,0.96,0.96,0.96,0.18
DecisionTreeClassifier,0.96,0.96,0.96,0.96,0.03
ExtraTreeClassifier,0.88,0.88,0.88,0.88,0.02
GaussianNB,0.81,0.81,0.81,0.81,0.02


In [ ]:
#clf_laz_str = LazyClassifier(verbose=0, ignore_warnings=True)
#lazy_model_str, lazy_pred_str = clf_laz_str.fit(X_train_scaled, X_test_scaled, y_train, y_test)
#lazy_model_str